## Model Description
The model used has 3 output branches, a decoder branch, a branch for classifying complex labels, and another branch for classifying all other disease. 
The decoder branch helps to ensure that model continously learns relevant features, even when the classifier loss is platueing. 
Complex label has a separate branch since complex labelled (more than 3 disease) images share features from all other disease, and thus it is classified separately to ensure it doesn't hurt the other class labels. 

## Importing Libraries

In [ ]:
!pip install -q efficientnet

In [ ]:
from kaggle_datasets import KaggleDatasets

import cv2
import json
import pickle
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import efficientnet.tfkeras as efn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

#physical_devices = tf.config.list_physical_devices('GPU') 
#try: 
#    tf.config.experimental.set_memory_growth(physical_devices[0], True)
#except:
#    print("Error")

## Initializing the validating the allocation of TPU (And the no of replicas)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    
print(f"Running on {strategy.num_replicas_in_sync} replicas")

## Functions for minor Data Augmentations and Image Resizing

In [ ]:
def decode_image(path, label = None, target_size = (512, 512)):
    
    img = tf.image.decode_jpeg(tf.io.read_file(path), channels = 3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize_with_pad(img, target_width=target_size[1], target_height=target_size[0])
    
    output_path = label['decoder_output']
    output_img = tf.image.decode_jpeg(tf.io.read_file(output_path), channels = 3)
    output_img = tf.cast(output_img, tf.float32) / 255.0
    output_img = tf.image.resize_with_pad(output_img, target_width=546, target_height=546)
    output_img = tf.image.per_image_standardization(output_img)
    label['decoder_output'] = output_img
    return img if label is None else img, label

def data_augment(img, label = None):
    seed = np.random.randint(low = 0, high = 1024)
    img = tf.image.random_flip_left_right(img, seed = seed)
    img = tf.image.random_flip_up_down(img, seed = seed)
    #img = tf.image.random_brightness(img, max_delta = 0.5)
    img = tf.image.random_crop(img, size = [512, 512, 3], seed = seed)
    
    output_img = label['decoder_output']
    output_img = tf.image.random_flip_left_right(output_img, seed = seed)
    output_img = tf.image.random_flip_up_down(output_img, seed = seed)
    #img = tf.image.random_brightness(img, max_delta = 0.5)
    output_img = tf.image.random_crop(output_img, size = [546, 546, 3], seed = seed)
    label['decoder_output'] = output_img

    return img if label is None else img, label

## TPU-specific Hyperparameters

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = strategy.num_replicas_in_sync * 32
GCS_DS_PATH = KaggleDatasets().get_gcs_path('plant-pathology-2021-fgvc8')
#GCS_DS_PATH = KaggleDatasets().get_gcs_path('aug-data-zip')

## Loading the data

In [ ]:
load_dir = "../input/plant-pathology-2021-fgvc8/"
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
#df = pd.read_pickle('../input/m1-aug-final/aug_images.csv')
#df['labels'] = df['labels'].apply(lambda x: ' '.join(x))
#df['img_path'] = df['img_path'].apply(lambda x: x.strip('./'))
#df['paths'] = df['img_path'].apply(lambda x: "../input/m1-aug-final/" + x)
#df.head()
df['paths'] = GCS_DS_PATH + "/train_images/" + df.image#"#G
sub_df = pd.read_csv(load_dir + 'sample_submission.csv')

In [ ]:
df['labels'] = df['labels'].apply(lambda x: x.split(" "))
df['complex'] = df['labels']
df['complex'] = df['complex'].apply(lambda x: 1 if 'complex' in x else 0)
df['scab'] = df['labels'].apply(lambda x: 1 if 'scab' in x else 0)
df['healthy'] = df['labels'].apply(lambda x: 1 if 'healthy' in x else 0)
df['frog_eye_leaf_spot'] = df['labels'].apply(lambda x: 1 if 'frog_eye_leaf_spot' in x else 0)
df['rust'] = df['labels'].apply(lambda x: 1 if 'rust' in x else 0)
df['powdery_mildew'] = df['labels'].apply(lambda x: 1 if 'powdery_mildew' in x else 0)

In [ ]:
scab_counter = 0
healthy_counter = 0
fgls_counter = 0
rust_counter = 0
powdery_mildew_counter = 0
for i in df.index:
    if 'scab' in df['labels'][i]:
        scab_counter+=1
    if 'healthy' in df['labels'][i]:
        healthy_counter+=1
    if 'frog_eye_leaf_spot' in df['labels'][i]:
        fgls_counter+=1
    if 'rust' in df['labels'][i]:
        rust_counter+=1
    if 'powdery_mildew' in df['labels'][i]:
        powdery_mildew_counter+=1
        
print("Scab -> {}".format(scab_counter))
print("Healthy -> {}".format(healthy_counter))
print("Grog Eye Leaf Spot -> {}".format(fgls_counter))
print("Rust -> {}".format(rust_counter))
print("Powdery Mildew -> {}".format(powdery_mildew_counter))

In [ ]:
scab_weight = len(df['labels']) / (6 * scab_counter)
healthy_weight = len(df['labels']) / (6 * healthy_counter)
fgls_weight = len(df['labels']) / (6 * fgls_counter)
rust_weight = len(df['labels']) / (6 * rust_counter)
powder_mildew_weight = len(df['labels']) / (6 * powdery_mildew_counter)


n_scab_weight = len(df['labels']) / (6 * (len(df['labels']) - scab_counter))
n_healthy_weight = len(df['labels']) / (6 * (len(df['labels']) - healthy_counter))
n_fgls_weight = len(df['labels']) / (6 * (len(df['labels']) - fgls_counter))
n_rust_weight = len(df['labels']) / (6 * (len(df['labels']) - rust_counter))
n_powder_mildew_weight = len(df['labels']) / (6 * (len(df['labels']) - powdery_mildew_counter))

In [ ]:
for i in df.index:
    if 'complex' in df['labels'][i]:
        df['labels'][i].remove('complex')

## Encoding Labels

In [ ]:
mlb = MultiLabelBinarizer()
df['class_labels'] = list(mlb.fit_transform(df['labels']))
mlb.classes_

In [ ]:
class_weights = {'positive_weights' : {0: fgls_weight,
                                       1: healthy_weight,
                                       2: powder_mildew_weight,
                                       3: rust_weight,
                                       4: scab_weight},
                'negative_weights': {0: n_fgls_weight,
                                     1: n_healthy_weight,
                                     2: n_powder_mildew_weight,
                                     3: n_rust_weight,
                                     4: n_scab_weight}}

print(class_weights)

In [ ]:
class_weights_complex = {'complex_output': {0: 0.56525696, 1: 4.33100883}}
df['class_labels'] = df['class_labels'].apply(lambda x: np.float32(x))

In [ ]:
df.head()

## Train Test Split

In [ ]:
train_df, valid_df = train_test_split(df, test_size = 0.2, random_state = 1, stratify = df['labels'])

train_dataset = (tf.data.Dataset
                 .from_tensor_slices((train_df.paths, {"other_disease_output": list(train_df['class_labels']),
                                                        #"complex_output": list(train_df['complex']),
                                                        "decoder_output": train_df.paths}))
                 .map(decode_image, num_parallel_calls = AUTO)#.cache()
                 .map(data_augment, num_parallel_calls = AUTO).batch(BATCH_SIZE).prefetch(1).repeat()
                 .shuffle(1024))

valid_dataset = (tf.data.Dataset
                 .from_tensor_slices((valid_df.paths, {"other_disease_output": list(valid_df['class_labels']),
                                                        #"complex_output": list(valid_df['complex']),
                                                        "decoder_output": valid_df.paths}))
                 .map(decode_image, num_parallel_calls=AUTO)#.cache()
                 .map(data_augment, num_parallel_calls = AUTO)#.repeat()
                 .batch(BATCH_SIZE).prefetch(1))

In [ ]:
def show(objs, titles=None, structure='LR', figsize=(5, 5), clims=False, axes=False, xlabels=False, title_colors=False):
    '''
    utility function to show plots (subplots)
    '''
    plt.style.use('default')
    plt.figure(figsize=figsize)
    for i, obj in enumerate(objs):
        if structure == 'LR': plt.subplot(1, len(objs), i+1)
        elif structure == 'TB': plt.subplot(len(objs), 1, i+1)
        plt.imshow(obj)
        #
        if axes: plt.axis(axes[i])
        else: plt.axis('off')
        #
        if xlabels: plt.xlabel(xlabels[i])
        #
        if clims: plt.clim(clims[i])
        #
        if titles:
            plt.title(titles[i], color=title_colors[i] if title_colors else 'teal')
    plt.show()

In [ ]:
for x,y in train_dataset.as_numpy_iterator():
    for i in range(5):
        label_img = mlb.classes_[np.argmax(y['other_disease_output'][i])]
        show(
            [x[i], y['decoder_output'][i]],
            [label_img, y['other_disease_output'][i]],
            clims=[[0, 1], [0, 1]],
            axes=['image', 'image'],
            figsize=((10, 5))
        )
        #show(x[:5], titles = np.argmax(y['other_disease_output']), figsize=(20,5))
    #show(y['decoder_output'][:5], figsize=(20,5))
    break

## Build Model

In [ ]:
#He Uniform Initializer for Dense Layer
import tensorflow as tf
def my_init(shape, dtype=None):
    initializer = tf.keras.initializers.he_uniform(seed = 1)
    return initializer(shape, dtype=dtype)

def conv2d(x, filters, filter_size, stride=(1,1), pad='same', name=None):
    x = tf.keras.layers.Conv2D(filters, filter_size, strides = stride, padding=pad, activation = 'relu', name=name, kernel_initializer = my_init)(x)
    x = tf.keras.layers.BatchNormalization(name = name+'_bn')(x)
    #x = tf.keras.layers.Activation('relu', name = name+'_activation')(x)
    return x

In [ ]:
def inception_b(x, name=None):
    #Branch 0
    # Conv2D(1,1) -> Conv2D(7,1) -> Conv2D(1,7) -> -> Conv2D(7,1) -> Conv2D(1,7)
    branch0 = conv2d(x, 192, (1,1), name = name+'_b0_0')
    branch0 = conv2d(branch0, 192, (7,1), name = name+'_b0_1')
    branch0 = conv2d(branch0, 224, (1,7), name = name+'_b0_2')
    branch0 = conv2d(branch0, 224, (7,1), name = name+'_b0_3')
    branch0 = conv2d(branch0, 256, (1,7), name = name+'_b0_4')
    
    #Branch 1
    # Conv2D(1,1) -> Conv2D(7,1) -> Conv2D(1,7)
    branch1 = conv2d(x, 192, (1,1), name = name+'_b1_0')
    branch1 = conv2d(branch1, 224, (7,1), name = name+'_b1_1')
    branch1 = conv2d(branch1, 256, (1,7), name = name+'_b1_2')
    
    #Branch 2
    # AvgPool(3,3) -> Conv2D(1,1)
    branch2 = tf.keras.layers.AveragePooling2D((1,1), padding='same', name = name+'_b3_0')(x)
    branch2 = conv2d(branch2, 128, (1,1), name = name+'_b3_1')
    
    #Branch 3
    # Conv2D(1,1)
    branch3 = conv2d(x, 384, (1,1), name = name+'_b4_0')
    
    #Concatenation
    x = tf.keras.layers.Concatenate(axis=3, name = name+'_concat')([branch0, branch1, branch2, branch3])
    
    return x

In [ ]:
def reduction_b(x, name=None):
    #Branch 0
    # Conv2D(1,1) -> Conv2D(1,7) -> Conv2D(7,1) -> -> Conv2D(3,3)
    branch0 = conv2d(x, 256, (1,1), name = name+'_b0_0')
    branch0 = conv2d(branch0, 256, (1,7), name = name+'_b0_1')
    branch0 = conv2d(branch0, 320, (7,1), name = name+'_b0_2')
    branch0 = conv2d(branch0, 320, (3,3), stride = (2,2), pad = 'valid', name = name+'_b0_3')
    
    #Branch 1
    # Conv2D(1,1) -> Conv2D(3,3)
    branch1 = conv2d(x, 192, (1,1), name = name+'_b1_0')
    branch1 = conv2d(branch1, 192, (3,3), stride = (2,2), pad = 'valid', name = name+'_b1_1')
    
    #Branch 2
    # AvgPool(3,3) -> Conv2D(1,1)
    branch2 = tf.keras.layers.MaxPooling2D((3,3), strides = (2,2), padding='valid', name = name+'_b3_0')(x)

    #Concatenation
    x = tf.keras.layers.Concatenate(axis=3, name = name+'_concat')([branch0, branch1, branch2])
    
    return x

In [ ]:
def get_model_x(input_shape):
    
    #x = tf.keras.applications.Xception(include_top = False, weights = 'imagenet', input_shape = input_shape)
    x = efn.EfficientNetB0(weights='noisy-student', include_top=False, pooling=None, input_shape=input_shape)
    
    #Branch Complex
    #cmplex = inception_b(x.output, name = 'complex_inception_c')
    #cmplex = reduction_b(x.output, name = 'complex_reduction')
    #cmplex_pool = tf.keras.layers.GlobalAveragePooling2D(name = 'complex_avg_pooling')(cmplex)
    
    
    #add = tf.keras.layers.Add(name = 'complex_other_disease_concat_1')([other_disease, cmplex_pool])
    
    #cmplex = tf.keras.layers.Dense(512, activation = 'relu', kernel_initializer = my_init, name = 'complex_dense0')(add)
    #cmplex = tf.keras.layers.Dense(16, activation = 'relu', kernel_initializer = my_init, name = 'complex_dense1')(cmplex)
    #cmplex = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'complex_output')(cmplex)
    
    #Branch Others
    #other_disease = tf.keras.layers.Add(name = 'complex_other_disease_concat')([other_disease, cmplex_pool])
    other_disease = reduction_b(x.output, name = 'other_disease_reduction')
    other_disease = tf.keras.layers.GlobalAveragePooling2D(name = 'other_disease_avg_pooling')(other_disease)
    other_disease = tf.keras.layers.Dense(512, activation = 'relu', kernel_initializer = my_init, name = 'other_disease_dense0')(other_disease)
    other_disease = tf.keras.layers.Dense(32, activation = 'relu', kernel_initializer = my_init, name = 'other_disease_dense1')(other_disease)
    other_disease = tf.keras.layers.Dense(5, activation = 'sigmoid', name = 'other_disease_output')(other_disease)
    
    #Branch Decoder
    #fgls = inception_c(x.output, name = 'fgls_inception_c')
    #decoder = tf.keras.layers.Conv2DTranspose(1024, (3,3),  strides=2, activation='relu', padding='same')(x.output)
    #decoder = tf.keras.layers.Conv2DTranspose(512, (3,3),  strides=2, activation='relu', padding='same')(decoder)
    #decoder = tf.keras.layers.Conv2DTranspose(128, (3,3),  strides=2, activation='relu', padding='same')(decoder)
    #decoder = tf.keras.layers.Conv2DTranspose(16, (3,3),  strides=2, activation='relu', padding='same')(decoder)
    #decoder = tf.keras.layers.Conv2DTranspose(3, (3,3),  strides=2, activation='sigmoid', padding='same', name = 'decoder_output')(decoder)
    # Decoder
    dec = tf.keras.layers.Conv2D(168, (1, 1), activation='relu', kernel_initializer = my_init)(x.output)
    dec = tf.keras.layers.Conv2DTranspose(144, (3, 3), strides=(2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(144, (3, 3), strides=(2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(112, (3, 3), strides=(2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(112, (3, 3), strides=(2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(56, (3, 3), strides=(2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(28, (3, 3), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(14, (2, 2), activation='relu', kernel_initializer = my_init)(dec)
    dec = tf.keras.layers.Conv2DTranspose(3, (1, 1), activation='tanh', name='decoder_output')(dec)
    
    model = tf.keras.models.Model(inputs = x.input, outputs = [other_disease, dec])
    return model

In [ ]:
#with strategy.scope():
model = get_model_x((512, 512, 3))
#model.summary()

In [ ]:
#model.load_weights('../input/decoder-branch-v7/decoder_multi_label_v7.h5')

In [ ]:
tf.keras.utils.plot_model(model, expand_nested=True)

In [ ]:
def custom_loss_(x_labels, y_labels):
        _mse_loss = tf.keras.losses.mean_squared_error(x_labels, y_labels)
        _mlse_loss = tf.keras.losses.mean_squared_logarithmic_error(x_labels, y_labels)
        _mae_loss = tf.keras.losses.mean_absolute_error(x_labels, y_labels)
        return 1. * _mse_loss + .1 * _mae_loss  + .1 * _mlse_loss

In [ ]:
from tensorflow.keras import backend as K

Wp = class_weights['positive_weights']
Wn = class_weights['negative_weights']

def custom_loss(y_true, y_logit):
    '''
    Multi-label cross-entropy
    * Required "Wp", "Wn" as positive & negative class-weights
    y_true: true value
    y_logit: predicted value
    '''
    loss = 0.#float(0)
    
    for i, key in enumerate(Wp.keys()):
        first_term = Wp[key] * y_true[i] * K.log(y_logit[i] + K.epsilon())
        second_term = Wn[key] * (1 - y_true[i]) * K.log(1 - y_logit[i] + K.epsilon())
        loss -= (first_term + second_term)
    return loss

In [ ]:
from tensorflow_addons.metrics.f_scores import F1Score, FBetaScore
with strategy.scope():
    model.compile(loss = {"other_disease_output": custom_loss, #tf.keras.losses.BinaryCrossentropy(), 
                          #"complex_output": tf.keras.losses.BinaryCrossentropy(), 
                          "decoder_output": tf.keras.losses.MeanAbsoluteError()},
                  loss_weights = {"other_disease_output": 1.0, #tf.keras.losses.BinaryCrossentropy(), 
                          #"complex_output": 1.5, 
                          "decoder_output": 10.0},
                  optimizer = tf.keras.optimizers.Adam(lr = 0.001, decay = 1e-5),
                  metrics = {"other_disease_output": ['acc', F1Score(len(mlb.classes_))]})#, #tf.keras.losses.BinaryCrossentropy(), 
                          #"complex_output": ['acc', F1Score(1)]})

checkpoint = tf.keras.callbacks.ModelCheckpoint('custom.h5', save_best_only = True, monitor = 'val_loss', mode = 'min')
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, patience = 3, mode = 'min', verbose = 1, restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 1, min_delta = 0.001, mode = 'min')

steps_per_epoch = train_df.shape[0] // BATCH_SIZE
valid_steps = valid_df.shape[0] //BATCH_SIZE

In [ ]:
h = model.fit(train_dataset, 
              epochs = 20, 
              validation_data = valid_dataset, 
              callbacks = [checkpoint, early_stop, reduce_lr], 
              steps_per_epoch = steps_per_epoch, 
              validation_steps = valid_steps)

In [ ]:
model.save('decoder_multi_label_v9.h5')

## Visualising Learning Plots

In [ ]:
plt.style.use("ggplot")
plt.figure()

plt.plot(h.history["loss"][1:], label="train_loss")
plt.plot(h.history["val_loss"][1:], label="val_loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper left")
plt.show()

In [ ]:
"""plt.style.use("ggplot")
plt.figure()

plt.plot(h.history["complex_output_loss"], label="train_loss")
plt.plot(h.history["val_complex_output_loss"], label="val_loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper left")

plt.show()"""

In [ ]:
plt.style.use("ggplot")
plt.figure()

plt.plot(h.history["other_disease_output_loss"][1:], label="train_loss")
plt.plot(h.history["val_other_disease_output_loss"][1:], label="val_loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(h.history["decoder_output_loss"][1:], label="train_loss")
plt.plot(h.history["val_decoder_output_loss"][1:], label="val_loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(loc="upper left")
plt.show()

In [ ]:
"""plt.plot(h.history["complex_output_acc"], label="train_acc")
plt.plot(h.history["val_complex_output_acc"], label="val_acc")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")
plt.show()"""

In [ ]:
plt.plot(h.history["other_disease_output_acc"], label="train_acc")
plt.plot(h.history["val_other_disease_output_acc"], label="val_acc")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(h.history["other_disease_output_f1_score"], label="train_F1_Score")
plt.plot(h.history["val_other_disease_output_f1_score"], label="val_F1_Score")
plt.title("F1 Score")
plt.xlabel("Epoch")
plt.ylabel("F1 Score")
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.plot(h.history["complex_output_f1_score"], label="train_F1_Score")
plt.plot(h.history["val_complex_output_f1_score"], label="val_F1_Score")
plt.title("F1 Score")
plt.xlabel("Epoch")
plt.ylabel("F1 Score")
plt.legend(loc="upper left")
plt.show()

## Confusion Matrix

In [ ]:
#model = tf.keras.models.load_model('../input/decoder-branch-model-v4/decoder_multi_label_v3.h5', compile=False, custom_objects={'my_init':my_init})#, 'custom_loss': custom_loss})

In [ ]:
valid_steps = valid_df.shape[0] //BATCH_SIZE
print(valid_steps)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix

In [ ]:
valid_df = valid_df.reset_index()

In [ ]:
df = df.reset_index()

In [ ]:
#for i in df.index:
#    if df['complex'][i] == 1.0:
#        print(df['class_labels'][i], '\t', df['labels'][i])

In [ ]:
output_other, output_complx,output_img = model.predict(valid_dataset, steps = valid_steps, verbose = 1)

In [ ]:
copy_other_disease = output_other.copy()
copy_output_complx = output_complx.copy()

In [ ]:
for i, x in enumerate(output_other):
    #num_output = np.round(x)
    if x[4] >= 0.5:
        output_other[i][4] = 1.0
    else:
        output_other[i][4] = 0.0
        
    if x[1] >= 0.5:
        output_other[i][1] = 1.0
    else:
        output_other[i][1] = 0.0
            
    if x[0] >= 0.5:
        output_other[i][0] = 1.0
    else:
        output_other[i][0] = 0.0
            
    if x[3] >= 0.5:
        output_other[i][3] = 1.0
    else:
        output_other[i][3] = 0.0
            
    if x[2] >= 0.5:
        output_other[i][2] = 1.0
    else:
        output_other[i][2] = 0.0

        
for i, x in enumerate(output_complx):
    if x >= 0.5:
        output_complx[i] = 1.0
    else:
        output_complx[i] = 0.0

In [ ]:
print("---Predicted--- \t ---True Label---\tComplex Predicition")
for i, x in enumerate(output_other):
    if valid_df['class_labels'][i].sum() == 0.0:
        pass
        #print(output_other[i], '\t',  valid_df['class_labels'][i], '\t', output_complx[i])

In [ ]:
valid_df['complex'][:176][0]

In [ ]:
print("---Predicted--- \t ---True Label---\tComplex Predicition")
for i, x in enumerate(output_other):
    if valid_df['complex'][i] == 1.0:
        pass
        #print(output_other[i], '\t',  valid_df['class_labels'][i], '\t', output_complx[i])

In [ ]:
true_other = 0
false_other = 0
true_cmplx = 0
false_cmplx = 0
for i in range(len(output_other)):
    #print(output_other[i], '\t',  valid_df['class_labels'][i], '\t', output_complx[i])
    if set(output_other[i]) == set(valid_df['class_labels'][i]):
        true_other += 1
    else:
        false_other += 1
    if output_complx[i][0] == valid_df['complex'][i]:
        true_cmplx += 1
    else:
        false_cmplx += 1

In [ ]:
valid_df.to_csv('valid_df.csv')

In [ ]:
print(true_other)
print(false_other)
print(true_cmplx)
print(false_cmplx)

In [ ]:
len(output_complx)

In [ ]:
def plot_matrix(cm, class_names):
    plt.style.use('default')
    #cm = confusion_matrix(x_labels, y_labels)
    fig, ax = plt.subplots(figsize=(5, 5))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.set(xticks=np.arange(cm.shape[1]),
            yticks=np.arange(cm.shape[0]),
            xticklabels=class_names,
            yticklabels=class_names,
            title='Confusion matrix',
            ylabel='True label',
            xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha='center', va='center',
                    color='white' if cm[i, j] > cm.max() / 2. else 'black')
    fig.tight_layout()
    plt.show()

In [ ]:
otehr_cm = multilabel_confusion_matrix(valid_df['class_labels'][:928].to_list(), output_other)

In [ ]:
#cm = np.asarray(cm, dtype=np.uint8)
plot_matrix(otehr_cm[0], ['Frog Eye', 'Not Frog Eye'])

In [ ]:
plot_matrix(otehr_cm[1], ['Heathly', 'Not Heathly'])

In [ ]:
plot_matrix(otehr_cm[2], ['Powdery Mildew', 'Not Powdery Mildew'])

In [ ]:
plot_matrix(otehr_cm[3], ['Rust', 'Not Rust '])

In [ ]:
plot_matrix(otehr_cm[4], ['Scab', 'Not Scab'])

In [ ]:
cmplx_cm = confusion_matrix(valid_df['complex'][:928], output_complx)

In [ ]:
plot_matrix(cmplx_cm, ['Complex', 'Not Complex'])